<a href="https://colab.research.google.com/github/JBLarpan/First-Contribution/blob/master/take_plt_in_folder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [134]:
!pip install Pillow
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib pydrive



In [135]:
from google.colab import drive
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

In [136]:
# Import the necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
from PIL import Image
from IPython.display import display, Javascript
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload



In [137]:
# Authenticate and mount Google Drive
auth.authenticate_user()
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [138]:
# Load the Iris dataset
iris_data_url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv"
df = pd.read_csv(iris_data_url)

In [139]:

# Display the first few rows of the dataset
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [140]:
# Calculate summary statistics
summary_stats = df.describe()
print(summary_stats)

       sepal_length  sepal_width  petal_length  petal_width
count    150.000000   150.000000    150.000000   150.000000
mean       5.843333     3.057333      3.758000     1.199333
std        0.828066     0.435866      1.765298     0.762238
min        4.300000     2.000000      1.000000     0.100000
25%        5.100000     2.800000      1.600000     0.300000
50%        5.800000     3.000000      4.350000     1.300000
75%        6.400000     3.300000      5.100000     1.800000
max        7.900000     4.400000      6.900000     2.500000


In [141]:
def print_scatter(df):
  sns.pairplot(df, hue="species")


In [142]:
#from IPython.display import Image as IPImage, display

def capture_screenshot(iteration, fig):
    # Capture the current plot using matplotlib
    #fig = plt.figure()
   # plt.axis('off')
    plt.savefig(f'/content/screenshot_folder/iteration_{iteration}.png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # Display the saved screenshot
    #save_path = f'/content/screenshot_folder/iteration_{iteration}.png'
   # display(IPImage(save_path))


In [143]:
#save images into drive bring their public url in a list
import time
def get_image_url(folder_id, iteration, image_file_path, url_list):
    drive_service = build('drive', 'v3')
    file_metadata = {
        'name': f'iteration_{iteration}.png',
        'parents': [folder_id],
        'mimeType': 'image/png',
        'role': 'viewer',
        'type': 'anyone'
    }
    media = MediaFileUpload(image_file_path, mimetype='image/png', resumable=True)
    uploaded_file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    image_id = uploaded_file['id']
    IMAGE_URL ='https://drive.google.com/uc?export=view&id='+image_id
    time.sleep(20)
    url_list.append(IMAGE_URL)
    #return IMAGE_URL


In [144]:
def genearate_url():
 n = 10 # Number of iterations
 url_list =[]
 for i in range(n):
   print(i)
   fig = print_scatter(df)
   capture_screenshot(i + 1, fig)
   image_file_path = f'/content/screenshot_folder/iteration_{i+1}.png'
   destination_folder_id = '1sesXPHJAmo9QFeMhEPGia34X2bZaUns7'
   get_image_url(destination_folder_id, i+1, image_file_path, url_list)

 print(url_list)
 return url_list




Preparing the slide ....

In [145]:
# Set the path to the folder containing the generated images
#images_folder_path = '/content/screenshot_folder'

In [146]:
# Set the Google Slide presentation ID
presentation_id = '1JqzW5nNoRzS8DER3Z892-xQNPHfdYTVseyV5UOILmwk'

# Set the starting iteration number
start_iteration = 1

# Build the Google Slides API service
service = build('slides', 'v1')

# Get the list of image files in the folder
#import os
#image_files = [f for f in os.listdir(images_folder_path) if os.path.isfile(os.path.join(images_folder_path, f))]


In [147]:
#crete a deck....

def create_deck():

 body = {
            'title': 'All images inserted into position.'
        }
 presentation = service.presentations() \
            .create(body=body).execute()
 print(f"Created presentation with ID:"
              f"{(presentation.get('presentationId'))}")

 presentation_id = presentation.get('presentationId')
 url_list = genearate_url()
 print(url_list)
 create_slides(presentation_id, url_list)


In [148]:
# Iterate over the image files and create slide one by one.
import time
def create_slides(presentation_id,url_list):
 i=0

 for url in url_list:
    # Generate the slide page with image and iteration label
    i= i+1
    requests = [
            {
                'createSlide': {
                    'objectId': f'slide:{i}',
                    'insertionIndex': '1'

                }
            }
        ]
    body = {
            'requests': requests
        }
    response = service.presentations() \
            .batchUpdate(presentationId=presentation_id, body=body).execute()
    create_slide_response = response.get('replies')[0].get('createSlide')
    print(f"Created slide with ID:"
              f"{(create_slide_response.get('objectId'))}")
    page_id = create_slide_response.get('objectId')
    text_filed_id = insert_text(presentation_id, page_id, i)
    text_formatting(presentation_id, page_id, i+1, text_filed_id)
    time.sleep(20)
    insert_image_below_heading(presentation_id, page_id,url)







In [149]:
#insert heading into every slide

def insert_text(presentation_id, page_id, i):
  element_id = f'MyTextBox_10_{i}'
  pt350 = {
            'magnitude': 24,
            'unit': 'PT'
        }
  requests = [
            {
                'createShape': {
                    'objectId': element_id,
                    'shapeType': 'TEXT_BOX',
                    'elementProperties': {
                        'pageObjectId': page_id,
                        'size': {
                            'height': {
                            'magnitude': 100,
                            'unit': 'PT'
                        },
                        'width': {
                            'magnitude': 600,
                            'unit': 'PT'
                        }
                        },
                        'transform': {
                            'scaleX': 1,
                            'scaleY': 1,
                            'translateX': 20,
                            'translateY': 30,
                            'unit': 'PT'
                        }
                    }
                }
            },

            # Insert text into the box, using the supplied element ID.
            {
                'insertText': {
                    'objectId': element_id,
                    'insertionIndex': 0,
                    'text': f'Class: {i}'
                }

            }
        ]

        # Execute the request.
  body = {
            'requests': requests
        }
  response = service.presentations() \
            .batchUpdate(presentationId=presentation_id, body=body).execute()
  create_shape_response = response.get('replies')[0].get('createShape')
  print(f"Created textbox with ID:"
              f"{(create_shape_response.get('objectId'))}")
  text_filed_id = create_shape_response.get('objectId')
  return text_filed_id

In [150]:
#heading formatting

def text_formatting(presentation_id, page_id, i, text_filed_id):
    print(text_filed_id)
    requests = [

            {
                'updateTextStyle': {
                    'objectId': text_filed_id,

                    'style': {
                        'fontFamily': 'Arial',
                        'bold': True,
                        'fontSize': {
                            'magnitude': 24,
                            'unit': 'PT'
                        }

                    },
                    'fields': 'fontFamily,fontSize'
                }
            }
        ]

        # Execute the requests.
    body = {
            'requests': requests
        }
    response = service.presentations().batchUpdate(
            presentationId=presentation_id, body=body).execute()
    print(f"Updated the text style for shape with ID:{text_filed_id}")


In [151]:
#inserting image in every slide.
def insert_image_below_heading(presentation_id, page_id, url):
    requests = []
    image_id = url.split("id=")[1]

    IMAGE_URL = url
    print(IMAGE_URL)


    emu4M = {
        'magnitude': 4000000,
        'unit': 'EMU'
    }
    emu8M = {
        'magnitude': 8000000,
        'unit': 'EMU'
    }
    requests.append({
        'createImage': {
            'objectId': image_id,
            'url': IMAGE_URL,
            'elementProperties': {
                'pageObjectId': page_id,
                'size': {
                    'height': emu4M,
                    'width': emu8M
                },
                'transform': {
                    'scaleX': 1,
                    'scaleY': 1,
                    'translateX': 100000,  # Left margin
                    'translateY': 100000,  # Vertical position below heading
                    'unit': 'EMU'
                }
            }
        }
    })

    # Execute the request.
    body = {
        'requests': requests
    }
    response = service.presentations().batchUpdate(presentationId=presentation_id, body=body).execute()
    create_image_response = response.get('replies')[0].get('createImage')
    print(f"Created image with ID: {(create_image_response.get('objectId'))}")







In [152]:
create_deck()


Created presentation with ID:1w_d0ewp3oPzLiKj1r03-bAunqnU1BLiOUfA-mduw57Q
0
1
2
3
4
5
6
7
8
9
['https://drive.google.com/uc?export=view&id=1S1yi6eOGCZsmytfZaWXCMG0hOaW0BAOi', 'https://drive.google.com/uc?export=view&id=1vxpHguS5ZBJ7Ma_qLRZ_oGIcAZGKkXdl', 'https://drive.google.com/uc?export=view&id=1-WXTb54t49Jix4pfhTPZ3AldxRid2XSy', 'https://drive.google.com/uc?export=view&id=1A-pJ68aPFAUkXCnx7LDPpJFc8F7WSsGc', 'https://drive.google.com/uc?export=view&id=1CDxhdbY56VntjkWWgT7yGnhQyKUq69CR', 'https://drive.google.com/uc?export=view&id=1kSThmaxIlSpnzgnqNNJSX8zDJ0qcec49', 'https://drive.google.com/uc?export=view&id=1K-r_l2y6aCNg_Szz0EDIfike5REVOgtu', 'https://drive.google.com/uc?export=view&id=1oiLqRmNGFVKspmwrmyczcsKuj5yzklZr', 'https://drive.google.com/uc?export=view&id=1TWkd24JvgvJvJfmbp0uYzOU3vH_yJoJX', 'https://drive.google.com/uc?export=view&id=16iX9gLJPPSrr19k5eglw6tY0Vq6zxHuY']
['https://drive.google.com/uc?export=view&id=1S1yi6eOGCZsmytfZaWXCMG0hOaW0BAOi', 'https://drive.google.co